In [1]:
%pip install fastapi uvicorn
%pip install openai
%pip install python-dotenv
%pip install httpx


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3

def connect_db():
    conn = sqlite3.connect('./db.db')
    return conn

def create_tables():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT NOT NULL,
            password TEXT NOT NULL
        );
    """)
    cursor.execute(""" 
        CREATE TABLE IF NOT EXISTS history (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            title TEXT NOT NULL,
            description TEXT NOT NULL,
            category TEXT NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users (id)
        );
    """)
    conn.commit()
    conn.close()


In [3]:
async def insert_history(user_id, title, description, category):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO history (user_id, title, description, category)
        VALUES (?, ?, ?, ?)
    """, (user_id, title, description, category))
    id = cursor.lastrowid
    conn.commit()
    conn.close()
    return id


async def path_history(id, title, description, category):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        UPDATE history
        SET title = ?, description = ?, category = ?
        WHERE id = ?
    """, (title, description, category, id))
    conn.commit()
    conn.close()

async def delete_history(id):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        DELETE FROM history
        WHERE id = ?
    """, (id,))
    conn.commit()
    conn.close()

async def get_all_historys():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT * FROM history
    """)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    historys = [dict(zip(columns, row)) for row in rows]
    conn.close()
    return historys

async def insert_user(name, email, password):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO users (name, email, password)
        VALUES (?, ?, ?)
    """, (name, email, password))
    userID = cursor.lastrowid
    conn.commit()
    conn.close()
    return userID

async def path_user(id, name, email, password):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        UPDATE users
        SET name = ?, email = ?, password = ?
        WHERE id = ?
    """, (name, email, password, id))
    conn.commit()
    conn.close()

async def delete_user(id):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        DELETE FROM users
        WHERE id = ?
    """, (id,))
    conn.commit()
    conn.close()

async def get_all_users():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT * FROM users
    """)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    users = [dict(zip(columns, row)) for row in rows]
    conn.close()
    return users

In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv

def get_chatgpt_response(prompt):
    load_dotenv()
    chave = os.getenv("API_KEY")

    client = OpenAI(
        api_key= chave,
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="gpt-3.5-turbo",
    )
    return chat_completion.choices[0].message.content


In [5]:
from pydantic import BaseModel

class UserCreate(BaseModel):
    name: str
    email: str
    password: str

class HistoryCreate(BaseModel):
    title: str
    description: str
    category: str
    userId: int

class HistoryGpt(BaseModel):
    prompt: str
    title: str
    category: str
    userId: int

In [6]:
from fastapi import FastAPI
import asyncio
import uvicorn

app = FastAPI() 

@app.on_event("startup")
async def startup():
    create_tables()
    
@app.post("/user")
async def create_user(UserCreate: UserCreate):
    userId = await insert_user(UserCreate.name, UserCreate.email, UserCreate.password)
    return {"message": f"User created successfully with ID {userId}"}

@app.patch("/user/{id}")
async def update_user(id: int, UserCreate: UserCreate):
    await path_user(id, UserCreate.name, UserCreate.email, UserCreate.password)
    return {"message": "User updated successfully"}

@app.delete("/user/{id}")
async def deleteUser(id: int):
    await delete_user(id)
    return {"message": "User deleted successfully"}

@app.get("/users")
async def getAllUsers():
    users = await get_all_users()
    return users

@app.post("/history")
async def create_history(HistoryCreate: HistoryCreate):
    historyId = await insert_history(HistoryCreate.userId, HistoryCreate.title, HistoryCreate.description, HistoryCreate.category)
    return {"message": f"History created successfully with ID {historyId}"}

@app.post("/gpthistory")
async def create_history_chatgpt(HistoryGpt: HistoryGpt):
    chatgpt_response = get_chatgpt_response(HistoryGpt.prompt)
    historyId =  await insert_history(HistoryGpt.userId, HistoryGpt.title, chatgpt_response, HistoryGpt.category)
    return {"message": f"Story updated successfully with ID {historyId}"}


@app.patch("/history/{id}")
async def update_history(id: int, HistoryCreate: HistoryCreate):
    await path_history(id, HistoryCreate.title, HistoryCreate.description, HistoryCreate.category)
    return {"message": "History updated successfully"}
     
@app.delete("/history/{id}")
async def deleteHistory(id: int):
    await delete_history(id)
    return {"message": "History deleted successfully with ID"}

@app.get("/historys")
async def getAllHistorys():
    historys = await get_all_historys()
    return historys


@app.get("/")
async def read_root():
    return {"Hello": "World"}


if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

C:\Users\jvcmo\AppData\Local\Temp\ipykernel_23512\3188813772.py:7: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
INFO:     Started server process [23512]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62855 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:62855 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:62856 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:62856 - "GET /openapi.json HTTP/1.1" 200 OK
